In [ ]:
import kagglehub
import os

path = kagglehub.dataset_download('rohitgrewal/airlines-flights-data')
print('Lokasi path berada di', path)
print('Isi path adalah', os.listdir(path))

100%|██████████| 2.33M/2.33M [00:00<00:00, 39.1MB/s]

Extracting files...


Lokasi path berada di /root/.cache/kagglehub/datasets/rohitgrewal/airlines-flights-data/versions/1
Isi path adalah ['airlines_flights_data.csv']


In [ ]:
import pandas as pd

df = pd.read_csv (path + "/airlines_flights_data.csv")
df

,index,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955
...,...,...,...,...,...,...,...,...,...,...,...,...
300148,300148,Vistara,UK-822,Chennai,Morning,one,Evening,Hyderabad,Business,10.08,49,69265
300149,300149,Vistara,UK-826,Chennai,Afternoon,one,Night,Hyderabad,Business,10.42,49,77105
300150,300150,Vistara,UK-832,Chennai,Early_Morning,one,Night,Hyderabad,Business,13.83,49,79099
300151,300151,Vistara,UK-828,Chennai,Early_Morning,one,Evening,Hyderabad,Business,10.00,49,81585


In [ ]:
df.drop(columns= 'index', inplace=True)
df.head(5)

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955


**PREPROCESSING**

In [ ]:
Features = ['source_city', 'departure_time','stops','arrival_time','destination_city','class','days_left']

In [ ]:
X = df[Features]
y = df['price']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
print(f"Jumlah data training {len(X_train)} dan jumlah data uji {len(X_test)}")

Jumlah data training 240122 dan jumlah data uji 60031


In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

scaler = StandardScaler()
num_feat = ['days_left']
num_transform = scaler

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
cat_feat =['source_city', 'departure_time', 'stops', 'arrival_time','destination_city']
cat_transform = encoder

In [ ]:
ordinal_feat = ['class']
class_categ = ['Economy','Business']
ordinal_transform = OrdinalEncoder(categories=[class_categ])

In [ ]:
from sklearn.preprocessing import FunctionTransformer
import numpy as np

log_transform = FunctionTransformer(np.log1p, inverse_func=np.expm1)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('log', log_transform),
        ('scaler', num_transform)
    ]),num_feat),
    ('ord', ordinal_transform, ordinal_feat),
    ('encoder', cat_transform, cat_feat)
], remainder ='drop')

In [ ]:
from sklearn.svm import SVR

SVR_Regre = SVR(kernel = 'rbf')

In [ ]:
from sklearn.compose import TransformedTargetRegressor

model_SVR = TransformedTargetRegressor(
    regressor = SVR_Regre,
    func = np.log1p,
    inverse_func= np.expm1
)

In [ ]:
svr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model_SVR)
])

**HYPERPARAMETER TUNING**

In [ ]:
params_dist = {
    'regressor__regressor__C': np.logspace(0, 2.5, 8),                  # dari 1 sampai 300
    'regressor__regressor__epsilon': np.linspace(0.05, 0.3, 5),        # margin toleransi
    'regressor__regressor__gamma': ['scale', 'auto']                    # Hanya digunakan oleh kernel non-linear
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_searchSVR = RandomizedSearchCV(
    estimator = svr_pipeline,
    param_distributions=params_dist,
    n_iter = 3,
    scoring = 'r2',
    cv = 2,
    verbose = 2,
    n_jobs = -1,
    random_state=42
)

In [ ]:
random_searchSVR.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits


KeyboardInterrupt: 

In [ ]:
best_svr = random_search.best_estimator_

**EVALUATE**

In [ ]:
print('Best Params ditemukan dengan RandomizedSearchCV :', random_search.best_params_)
print('Best CV Score ditemuakan dengan RandomizedSearchCV :', random_search.best_score_)

In [ ]:
print(best_svr)

In [ ]:
y_pred = best_svr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
MAE = mean_absolute_error(y_test, y_pred)
R2 = r2_score(y_test, y_pred)

print("=== Hasil Evaluasi Model ===")
print(f"Score R2 yang ditemukan :{R2:.4f}")
print(f"Score MAE yang ditemukan :{MAE:.4f}")
print(f"Score RMSE yang ditemukan {RMSE:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

seaborn_version = 'seaborn-v0_8-whitegrid'

plt.figure(figsize=(8,8))
plt.style.use(seaborn_version)

sns.scatterplot(x=y_test, y=y_pred, color = 'dodgerblue', alpha=0.7)
max_val = max(y_test.max(), y_pred.max())
y_pred.max()

plt.plot([0, max_val],[0, max_val], 'r--', lw=2, label='Garis prediksi sempurna (y=x)')
plt.title('Prediksi vs Nilai Aktual', fontsize=16, fontweight='bold')
plt.xlabel('Harga Aktual', fontsize=12)
plt.ylabel('Harga Prediksi', fontsize=12)
plt.legend()
plt.text(0.05,0.9, f'R2:{R2:.4f}', transform=plt.gca(). transAxes, fontsize=12, verticalalignment='top')
plt.show()

In [ ]:
residual = y_test - y_pred

plt.figure(figsize=(8,6))
plt.style.use(seaborn_version)

sns.histplot(residual, kde=True, color='coral', bins=30)
plt.title('Distribusi Residual Error', fontsize=16, fontweight='bold')
plt.xlabel('Residual (Harga Aktual - Harga Prediksi)', fontsize=12)
plt.axvline(0, color='r', linestyle='--', label='Pusat di Nol')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.style.use(seaborn_version)

sns.scatterplot(x=y_pred, y=residual, color='forestgreen', alpha=0.6)
plt.axhline(0, color='r', linestyle='--', lw=2)
plt.title('Residual vs Nilai Prediksi', fontsize=16, fontweight='bold')
plt.xlabel('Harga Prediksi', fontsize=12)
plt.ylabel('Residual', fontsize=12)
plt.show()


In [ ]:
# Mengambil step preprocessor dan regressor SVR

preprocessor = best_svr.named_steps['preprocessor']
model = best_svr.named_steps['regressor'].regressor_

In [ ]:
# Ambil fitur numerik
num_features = preprocessor.transformers_[0][2]

# Ambil fitur ordinal
ord_features = preprocessor.transformers_[1][2]

# Ambil fitur kategori (one-hot)
ohe = preprocessor.transformers_[2][1]
cat_features = preprocessor.transformers_[2][2]
ohe_features = list(ohe.get_feature_names_out(cat_features))


# Gabungkan semuanya
all_features = list(num_features) + list(ord_features) + ohe_features

In [ ]:
# Feature importance dari model SVR
importances = model.feature_importances_

# Buat dataframe
feature_importance_df = pd.DataFrame({
    'Feature': all_features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False).reset_index(drop=True)


In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=feature_importance_df.head(15), x='Importance', y='Feature')
plt.title('Top 15 Feature Importances - Support Vector Regression')
plt.xlabel('Importance Score')
plt.ylabel('Features Name')
plt.tight_layout()
plt.show()